In [17]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")
model.invoke("최근 로제가 발표한 신곡은 무엇인가요?")

AIMessage(content='최근 로제가 발표한 신곡은 "GONE"과 "ON THE GROUND"가 포함된 솔로 싱글 앨범인 "R"에 수록되어 있습니다. 이 앨범은 2021년 발매되었습니다. 그 이후의 신곡 정보는 확인이 필요할 수 있으니, 최신 업데이트를 위해 공식 소셜 미디어나 음악 스트리밍 플랫폼을 참고하시기 바랍니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 89, 'prompt_tokens': 19, 'total_tokens': 108, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BqVnF5w7lI7BBjIMGCcqbdGDoqvR2', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--0ee3da42-09fe-4b9f-9530-0ae9d7c527b3-0', usage_metadata={'input_tokens': 19, 'output_tokens': 89, 'total_tokens': 108, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [2]:
# 덕덕고 설치
%pip install -U duckduckgo-search

  Attempting uninstall: duckduckgo-search
    Found existing installation: duckduckgo_search 8.0.5
    Uninstalling duckduckgo_search-8.0.5:
      Successfully uninstalled duckduckgo_search-8.0.5
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# 덕덕고 검색 테스트
from langchain_community.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults(results_separator=';\n')
docs = search.invoke("최근 로제가 발표한 신곡은 무엇인가요?")

print(docs)

snippet: Nov 19, 2024 · 더블랙레이블은 로제가 오는 22일 오후 2시 (한국기준) 싱글 '넘버 원 걸 (number one girl)'을 선공개한다고 밝혔다. 이는 지난달 18일 싱글 '아파트' 선공개 이후 약 한 달 만의 …, title: 블랙핑크 로제, '아파트' 신화 잇는다... 22일 신곡 공개 - 한국일보, link: https://www.hankookilbo.com/News/Read/A2024111917190000213;
snippet: Dec 21, 2024 · 20일 (현지시간) 공개된 최신 차트에 따르면 로제가 팝스타 브루노 마스와 호흡을 맞춘 히트곡 '아파트' (APT.)는 영국 오피셜 싱글 차트 '톱 100'에서 전주보다 한 계단 하락한 …, title: 블랙핑크 로제, 솔로 1집으로 영국 싱글 5위·앨범 49위 - 연합뉴스, link: https://www.yna.co.kr/view/AKR20241221022400005;
snippet: Nov 22, 2024 · [조이뉴스24 정지원 기자] 블랙핑크 로제가 신곡 'number one girl'을 발표했다. 더블랙레이블은 22일 공식 SNS를 통해 로제의 새 싱글 'number one girl' 음원과 …, title: [오늘가요] "내가 특별하다고 말해줘"…로제, '넘버 원 걸' 발표, link: https://www.joynews24.com/view/1786189;
snippet: Nov 22, 2024 · 걸그룹 블랙핑크의 멤버 로제가 22일 새 싱글 ‘넘버 원 걸 (Number one girl)’을 발표했다. 넘버 원 걸은 로제가 내달 6일에 발매하는 첫 정규앨범 ‘로지 (rosie)’의 선공개 곡이다. …, title: 로제, 신곡 ‘넘버 원 걸’ 공개… 브루노 마스가 프로듀싱, link: https://biz.chosun.com/entertainment/enter_general/2024/11/22/Q6AJ6YGRUJDPNIRGX5BKHHUBBY/


In [18]:
# 확장 쿼리 구현
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "아래 context에 기반하여 사용자의 질문에 답변하라.:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

# LCEL 방식으로 바로 gpt와 연결
document_chain = question_answering_prompt | model

In [19]:
# 메시지 저장하고 답변 생성 후 출력하기
from langchain.memory import ChatMessageHistory

# 채팅 메시지를 저장할 메모리 객체 생성
chat_history = ChatMessageHistory()
# 사용자의 질문을 메모리에 저장
chat_history.add_user_message("최근 로제가 발표한 신곡은 무엇인가요?")

# 문서 검색하고 답변 생성
answer = document_chain.invoke(
    {
        "messages": chat_history.messages,
        "context": docs,
    }
)

# 생성된 답변을 메모리에 저장
chat_history.add_ai_message(answer)

print(answer)

content='죄송하지만 해당 정보를 가지고 있지 않습니다. 로제의 최근 신곡에 대한 업데이트는 공식 음악 플랫폼이나 로제의 공식 소셜 미디어 계정을 확인해 보시기를 권장합니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 494, 'total_tokens': 537, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BqVnRAhQLxZyS9EVYPb7tddsGs97A', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--0caa0ffb-caae-4f28-9cbb-18482fe2d73f-0' usage_metadata={'input_tokens': 494, 'output_tokens': 43, 'total_tokens': 537, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


### 검색 기능에 옵션 설정

In [5]:
# 최근 일주일 뉴스만 검색하기
# DuckDuckGo API 래퍼를 사용하여 검색할 때 검색 매개변수를 설정하는 클래스 import
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

# 한국 지역("kr-kr")을 기준, 최근 일주일("w") 내의 검색 결과를 가져오도록 초기화
wrapper = DuckDuckGoSearchAPIWrapper(region="kr-kr", time="w")

In [6]:
# 뉴스 소스 위주로 검색하도록 지정하기
# 검색 기능을 위한 DuckDuckGoSearchResults 초기화
from langchain_community.tools import DuckDuckGoSearchResults
search = DuckDuckGoSearchResults(
    api_wrapper=wrapper, # 앞에서 정의한 wrapper 사용
    source="news", 
    results_separator=";\n", # 결과 항목 사이에 구분자 사용 - 세미콜론과 줄 바꿈
)

In [7]:
# 검색 결과 반환, 출력
# "로제의 신곡 APT에 대한 반응"을 검색하고 결과를 docs에 저장
docs = search.invoke("한국의 딥러닝 모델")

# 검색 결과 출력
print(docs)

snippet: 4 days ago · 천문학적 비용을 들이지 않고 고성능 한국어 '거대 언어 모델 (LLM)’을 개발할 수 있다는 점이 입증됐다. LLM은 방대한 양의 텍스트 데이터를 학습해 인간 언어를 이해하고 …, title: 이재진 교수 연구팀, 고성능 한국어 LLM 개발 | 서울대학교 컴퓨터 ..., link: https://cse.snu.ac.kr/community/news/1191;
snippet: 4 days ago · 이번 연구는 영어 기반 LLM ‘Llama’를 바탕으로 3TB 규모의 한국어 웹 데이터를 수집·전처리하고, 연속 학습 및 사후 학습 기법을 적용해 중국의 ‘딥시크 (DeepSeek)’처럼 …, title: 이재진 교수 연구팀, 고성능 한국어 LLM ‘Llama-Thunder’ 개발 ..., link: https://gsds.snu.ac.kr/이재진-교수-연구팀-고성능-한국어-llm-llama-thunder-개발/;
snippet: 2 hours ago · 과학기술정보통신부 산하 한국전파진흥협회(RAPA)가 아마존웹서비스코리아(AWS)와 협력해 AI 인재 양성에 나선다. 협회는 클라우드, 컨테이너, …, title: 한국전파진흥협회, AWS와 ‘AI 스쿨’ 운영 AI 인재 양성, link: https://www.dt.co.kr/article/12002196;
snippet: 2 hours ago · [이슈앤비즈 김하성 기자]과학기술정보통신부 산하 한국전파진흥협회(RAPA)는 아마존웹서비스코리아(AWS)와 협력하여, 클라우드, 컨테이너, 딥러닝 기술을 통합해 AI …, title: 한국전파진흥협회、AWS와 협력해 'AWS AI School' 국내 최초 …, link: http://www.issuenbiz.com/news/articleView.html?idxno=63211


In [ ]:
# 특정 웹 사이트에서 검색하기 이거 안하는 게 좋겠다.
# ytn.co.kr 에서 검색
# docs = search.invoke("site:k-trendynews.com 로제의 노래 APT")
# print(docs)

### 기사 링크 가져오기

In [8]:
# 검색 결과의 링크들을 저장할 빈 리스트 초기화
links = []

# 검색 결과를 세미콜론과 줄 바꿈 기준으로 분리하고, 각 결과 항목에서 링크 추출
for doc in docs.split(";\n"):
    print(doc)
    link = doc.split("link:")[1].strip()
    links.append(link)

# 모든 링크를 출력
print(links)

snippet: 4 days ago · 천문학적 비용을 들이지 않고 고성능 한국어 '거대 언어 모델 (LLM)’을 개발할 수 있다는 점이 입증됐다. LLM은 방대한 양의 텍스트 데이터를 학습해 인간 언어를 이해하고 …, title: 이재진 교수 연구팀, 고성능 한국어 LLM 개발 | 서울대학교 컴퓨터 ..., link: https://cse.snu.ac.kr/community/news/1191
snippet: 4 days ago · 이번 연구는 영어 기반 LLM ‘Llama’를 바탕으로 3TB 규모의 한국어 웹 데이터를 수집·전처리하고, 연속 학습 및 사후 학습 기법을 적용해 중국의 ‘딥시크 (DeepSeek)’처럼 …, title: 이재진 교수 연구팀, 고성능 한국어 LLM ‘Llama-Thunder’ 개발 ..., link: https://gsds.snu.ac.kr/이재진-교수-연구팀-고성능-한국어-llm-llama-thunder-개발/
snippet: 2 hours ago · 과학기술정보통신부 산하 한국전파진흥협회(RAPA)가 아마존웹서비스코리아(AWS)와 협력해 AI 인재 양성에 나선다. 협회는 클라우드, 컨테이너, …, title: 한국전파진흥협회, AWS와 ‘AI 스쿨’ 운영 AI 인재 양성, link: https://www.dt.co.kr/article/12002196
snippet: 2 hours ago · [이슈앤비즈 김하성 기자]과학기술정보통신부 산하 한국전파진흥협회(RAPA)는 아마존웹서비스코리아(AWS)와 협력하여, 클라우드, 컨테이너, 딥러닝 기술을 통합해 AI …, title: 한국전파진흥협회、AWS와 협력해 'AWS AI School' 국내 최초 …, link: http://www.issuenbiz.com/news/articleView.html?idxno=63211
['https://cse.snu.ac.kr/community/news/1191', 'https://gsds.snu.ac.kr/이재진-교수-연구팀-고성능-한국어-llm-ll

In [9]:
# 웹 페이지 내용 비동기로 읽어 오기
# 랭체인의 WebBaseLoader를 사용하여 웹 페이지의 내용 불러오기
from langchain_community.document_loaders import WebBaseLoader

# WebBaseLoader 객체를 생성. 'links'는 웹 페이지의 URL 목록을 담고 있는 변수
# bs_get_text_kwargs는 BeautifulSoup의 get_text() 메서드에 전달될 추가 인자
loader = WebBaseLoader(
    web_path=links,
    bs_get_text_kwargs={
        "strip": True, # 웹 페이지에서 텍스트를 가져올 때 앞뒤의 공백 제거
    }
)

# 비동기로 웹 페이지의 내용을 로드하고, 각 문서를 page_contents 리스트에 추가
page_contents = [] # 페이지 내용을 저장할 리스트
async for doc in loader.alazy_load():
    page_contents.append(doc) # 불러온 문서를 page_contents 리스트에 추가

# page_contents에 있는 각 웹 페이지의 내용 출력
for content in page_contents:
    print(content)
    print('-------------------------')


USER_AGENT environment variable not set, consider setting it to identify your requests.
Fetching pages:   0%|          | 0/4 [00:00<?, ?it/s]Error fetching https://gsds.snu.ac.kr/이재진-교수-연구팀-고성능-한국어-llm-llama-thunder-개발/ with attempt 1/3: Cannot connect to host gsds.snu.ac.kr:443 ssl:default [현재 연결은 원격 호스트에 의해 강제로 끊겼습니다]. Retrying...
Error fetching https://cse.snu.ac.kr/community/news/1191 with attempt 1/3: Cannot connect to host cse.snu.ac.kr:443 ssl:default [현재 연결은 원격 호스트에 의해 강제로 끊겼습니다]. Retrying...
Fetching pages: 100%|##########| 4/4 [00:02<00:00,  1.54it/s]

page_content='이재진 교수 연구팀, 고성능 한국어 LLM 개발 | 서울대학교 컴퓨터공학부로그인ENGsearch소식arrow_forward_ios새 소식새 소식소식공지사항새 소식세미나신임교수초빙학생회About학부 소개교수진학부 안내대학원 안내Resources공지사항세미나시설 예약 안내Research신임교수초빙연구실 목록Top Conference List10-10 ProjectMore연합전공 인공지능(학사)지능형컴퓨팅사업단컴퓨터 연구소해동학술정보실개인정보처리방침|학부 연락처|찾아오시는 길08826 서울특별시 관악구 관악로 1 서울대학교 공과대학 컴퓨터공학부 행정실(301동 316호)Powered byCSEREAL© Department of CSE, SNU.All Rights Reserved.천문학적 비용을 들이지 않고 고성능 한국어 '거대 언어 모델(LLM)’을 개발할 수 있다는 점이 입증됐다. LLM은 방대한 양의 텍스트 데이터를 학습해 인간 언어를 이해하고 생성하는 인공지능(AI) 모델이다.이재진 서울대 데이터사이언스대학원 교수 연구팀이 영어 기반 언어 모델인 ‘라마(Llama)’를 개량해 한국어에 특화된 언어 모델인 ‘Llama-Thunder-LLM’, 한국어 전용 토크나이저 ‘Thunder-Tok’, 한국어 LLM 성능을 객관적으로 평가하는 ‘Thunder-LLM 한국어 벤치마크’를 개발하였다. 토크나이저는 문장을 언어 모델이 이해할 수 있는 단위인 ‘토큰’으로 쪼개는 도구이고 벤치마크는 특정 기준과 비교해 성능을 측정하고 평가하는 도구다.한국어에 특화된 LLM을 만들려면 개발에 막대한 비용이 든다. LLM 구축을 위한 데이터가 한정적이라는 한계도 있다. LLM이 대기업과 해외 빅테크 기업 중심으로 개발돼온 이유다. 중소 연구기관이나 대학에서는 LLM 연구개발을 하기 어렵다. 연구팀은 데이터 수집부터 사후 학습까지 언어 모델 학습의 모든 단계를 자체 진행해 중국 LLM ‘딥시크’처럼 제한된 자원으로 고성능 언어 모델을 구축할 수 있다는 점을 입증

### 뷰티풀수프를 이용해 특정 영역만 가져오기

In [14]:
# 주어진 URL에서 텍스트를 가져오는 함수 만들기
import requests
from bs4 import BeautifulSoup

def get_article_text(url):
    try:
        # URL에 GET 요청을 보냄
        response = requests.get(url)
        # 요청이 성공하지 못하면 예외를 발생
        response.raise_for_status()

        # BeautifulSoup를 사용하여 HTML 내용 파싱
        soup = BeautifulSoup(response.content, 'html.parser')

        # 클래스가 'stroy-new article'인 <article> 태그 찾기 (내가 찾은 기사에 맞게 조정)
        article = soup.find('div', class_='page-container-content')

        # 기사를 찾았다면 그 텍스트를 반환
        if article:
            return article.get_text(strip=True)
        else:
            try:
                # 기사에 해당되는 태그를 찾지 못했을 경우 첫번째 <article> 태그의 내용을 반환
                if soup.find('article'):
                    return soup.find('article').get_text(strip=True)
                elif soup.find("div", id="sun-editor-editable"):
                    return soup.find("div", id="sun-editor-editable").get_text(strip=True)
            except:
                return "기사 내용을 찾을 수 없습니다."
            
    except requests.exceptions.RequestException as e:
        return f"URL을 가져오는 중 오류 발생: {e}"

In [15]:
# 주어진 URL에서 텍스트 가져오기
articles = [] # 가져온 내용을 리스트에 담는 변수 선언
for link in links:
    print(f"URL: {link}\n")
    article_text = get_article_text(link)
    print(f"Content:\n{article_text}")
    print("---------------------------------------------")
    articles.append(article_text)

URL: https://cse.snu.ac.kr/community/news/1191

Content:
None
---------------------------------------------
URL: https://gsds.snu.ac.kr/이재진-교수-연구팀-고성능-한국어-llm-llama-thunder-개발/

Content:
(왼쪽부터) 손영준 서울대 데이터사이언스대학원 박사과정생, 소연경 박사과정생, 박찬우 석사과정생, 이재진 교수, 김진표 박사과정생, 석지헌 박사과정생, 조경제 박사과정생, 박종원 석사과정생, 김종민 박사과정생서울대 데이터사이언스대학원 이재진 교수 연구팀이 고성능 한국어 특화 거대언어모델(LLM) ‘Llama-Thunder-LLM’, 한국어 전용 토크나이저 ‘Thunder-Tok’, LLM 평가용 벤치마크 ‘Thunder-LLM’을 개발했습니다.이번 연구는 영어 기반 LLM ‘Llama’를 바탕으로 3TB 규모의 한국어 웹 데이터를 수집·전처리하고, 연속 학습 및 사후 학습 기법을 적용해 중국의 ‘딥시크(DeepSeek)’처럼 제한된 자원으로도 고성능 LLM을 개발할 수 있음을 입증했습니다.특히 한국어 문법 특성을 반영한 Thunder-Tok은 기존 대비 약 44%의 토큰 수 절감을 통해 학습 및 추론 효율을 크게 향상시켰으며, 자체 구축한 벤치마크를 통해 한국어 LLM의 성능을 객관적이고 체계적으로 평가할 수 있는 기반도 마련했습니다.연구 성과는 ‘초거대 AI모델 및 플랫폼 최적화 센터’웹페이지에 공개되었습니다.관련기사
---------------------------------------------
URL: https://www.dt.co.kr/article/12002196

Content:
URL을 가져오는 중 오류 발생: 403 Client Error: Forbidden for url: https://www.dt.co.kr/article/12002196
-----------------------------------------

In [20]:
# 답변 생성하기
chat_history.add_user_message("한국의 딥러닝 모델")

# 문서 검색하고 답변 생성
answer = document_chain.invoke(
    {
        "messages": chat_history.messages,
        "context": docs,
    }
)

chat_history.add_ai_message(answer)
print(answer)

content="최근 한국에서 고성능 딥러닝 모델이 개발되고 있습니다. 예를 들어, 서울대학교 이재진 교수 연구팀은 한국어를 위한 고성능 '거대 언어 모델 (LLM)'을 개발했습니다. 이 모델은 영어 기반 LLM인 'Llama'를 바탕으로 하여, 3TB 규모의 한국어 웹 데이터를 수집 및 전처리하고, 연속 학습 및 사후 학습 기법을 적용하여 구성되었습니다. 이와 같은 연구는 한국어 자연어 처리 분야에서 중요한 진전을 이룩하고 있습니다." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 124, 'prompt_tokens': 552, 'total_tokens': 676, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BqVniui8YQMBKjHXSKF73Z5YowSGw', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--9852114a-81ff-4990-b4d3-81619a11a09c-0' usage_metadata={'input_tokens': 552, 'output_tokens': 124, 'total_tokens': 676, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'a